# TRABAJO INTEGRADOR FINAL
## Aprendizaje de máquina II

### Optimización de hiperparámetros del modelo utilizando optuna

AUTOR: Juan Ignacio Ribet

FECHA: 04-Ago-2023


In [20]:
#Importo las librerias necesarias
import optuna
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso

Cargo el dataset para entrenamiento y test.

In [21]:
# Cargo los datasets de train y test
df_train = pd.read_csv(
    'C:/Users/juani/Documents/Especializacion IA/Aprendizaje de Maquina II/Aprendizaje_Maq_2_CEIA/results/outdata_train.csv', index_col=0)
df_test = pd.read_csv(
    'C:/Users/juani/Documents/Especializacion IA/Aprendizaje de Maquina II/Aprendizaje_Maq_2_CEIA/results/outdata_Test.csv', index_col=0)

seed = 28
# División de dataset de entrenaimento y validación
x_data = df_train.drop(columns='Item_Outlet_Sales')
x_train, x_val, y_train, y_val = train_test_split(x_data, df_train['Item_Outlet_Sales'], test_size=0.3, random_state=seed)

Defino la función para evaluar las métricas.

In [22]:
metrics_models = pd.DataFrame()

def evaluation_score(name, model):
    '''
    Función para evaluar los modelos y guardar resultados
    '''
    # Metricas de entrenamiento
    mse_train = round(metrics.mean_squared_error(y_train, model.predict(x_train))**0.5, 2)
    R2_train = round(model.score(x_train, y_train), 4)
    print(f'Métricas del Modelo: *{name}*')
    print(f'ENTRENAMIENTO: RMSE: {mse_train} - R2: {R2_train}')
    
    # Metricas de validacion
    mse_val = round(metrics.mean_squared_error(y_val, model.predict(x_val))**0.5, 2)
    R2_val = round(model.score(x_val, y_val), 4)
    print(f'VALIDACIÓN: RMSE: {mse_val} - R2: {R2_val}', '\n')

    # saves the metrics in the dataframe 'eval'
    global metrics_models
    metrics_model = pd.DataFrame({'mse_train': [mse_train], 'R2_train': [R2_train],
                                  'mse_val': [mse_val], 'R2_val': [R2_val]})
    metrics_model.rename(index=lambda x: name, inplace=True)
    try:
        metrics_models = metrics_models.drop(name)
    except:
        pass
    metrics_models = pd.concat([metrics_models, metrics_model], ignore_index=False)
    #metrics_models.sort_index()

Entreno el modelo original para comprar métricas.

In [23]:
# Entrenamiento del modelo
model = LinearRegression()
model.fit(x_train,y_train)

# Predicción del modelo ajustado para el conjunto de validación
pred = model.predict(x_val)

# Evaluo el modelo
evaluation_score('LinearRegression',model)

Métricas del Modelo: *LinearRegression*
ENTRENAMIENTO: RMSE: 1169.35 - R2: 0.5284
VALIDACIÓN: RMSE: 1146.66 - R2: 0.5526 



Con Optuna busco optimizar el modelo lineal con los métodos Ridge y Lasso y diferentes alphas (0.0 , 2.0)

In [24]:
def objective_linear_model(trial):
    """
    Objective function for optimizing hyperparameters of a linear model, Ridge & Lasso, using Optuna.
    
    Args:
        trial: A `Trial` object from Optuna that contains the state of the optimization trial.
    
    Returns:
        The mean_squared_error of the validation set using the hyperparameters suggested by Optuna.
    """
    
    # Seteo de los hiperparametros
    regression_method = trial.suggest_categorical('regression_method', ('ridge', 'lasso'))
    if regression_method == 'ridge':
        ridge_alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
        model = Ridge(alpha=ridge_alpha)
    else:
        lasso_alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
        model = Lasso(alpha=lasso_alpha)

    # Entrenamiento y evalución del modelo
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    mse_val = metrics.mean_squared_error(y_val, y_pred)**0.5

    # Metrica de evalucion por optuna
    return mse_val

study_linearModel = optuna.create_study(direction='minimize')
study_linearModel.optimize(objective_linear_model, n_trials=50)

# Imprimir los resultados de la optimización
print('Best trial: score {}, params {}'.format(study_linearModel.best_trial.value, study_linearModel.best_trial.params))

[I 2023-08-05 11:42:00,912] A new study created in memory with name: no-name-091deaeb-c79f-4cf2-8f44-cfff2116980f
[I 2023-08-05 11:42:01,033] Trial 0 finished with value: 1146.6394865677214 and parameters: {'regression_method': 'lasso', 'alpha': 0.3105269797816339}. Best is trial 0 with value: 1146.6394865677214.
[I 2023-08-05 11:42:01,073] Trial 1 finished with value: 1146.6447264208523 and parameters: {'regression_method': 'lasso', 'alpha': 0.5437124203125603}. Best is trial 0 with value: 1146.6394865677214.


[I 2023-08-05 11:42:01,105] Trial 2 finished with value: 1146.6618800776214 and parameters: {'regression_method': 'ridge', 'alpha': 0.1441532143816937}. Best is trial 0 with value: 1146.6394865677214.
[I 2023-08-05 11:42:01,153] Trial 3 finished with value: 1146.6262638364951 and parameters: {'regression_method': 'lasso', 'alpha': 1.4015239260335128}. Best is trial 3 with value: 1146.6262638364951.
[I 2023-08-05 11:42:01,185] Trial 4 finished with value: 1146.6698724129628 and parameters: {'regression_method': 'lasso', 'alpha': 0.8608613629051729}. Best is trial 3 with value: 1146.6262638364951.
[I 2023-08-05 11:42:01,218] Trial 5 finished with value: 1146.641716953571 and parameters: {'regression_method': 'lasso', 'alpha': 1.175392085528772}. Best is trial 3 with value: 1146.6262638364951.
[I 2023-08-05 11:42:01,242] Trial 6 finished with value: 1146.6478372780196 and parameters: {'regression_method': 'ridge', 'alpha': 0.8420307016392878}. Best is trial 3 with value: 1146.626263836495

Best trial: score 1146.6047498307905, params {'regression_method': 'lasso', 'alpha': 1.9985473385524772}


In [25]:
study_linearModel.best_trial.params['regression_method']

'lasso'

In [26]:
# Entreno el modelo optimo
model_op = Lasso(study_linearModel.best_trial.params['alpha'])
model_op.fit(x_train, y_train)

# Predicción del modelo ajustado para el conjunto de validación
pred = model_op.predict(x_val)

# Evaluo el modelo
evaluation_score('LinearModel_Optimizado',model_op)

metrics_models

Métricas del Modelo: *LinearModel_Optimizado*
ENTRENAMIENTO: RMSE: 1169.6 - R2: 0.5282
VALIDACIÓN: RMSE: 1146.6 - R2: 0.5526 



,mse_train,R2_train,mse_val,R2_val
LinearRegression,1169.35,0.5284,1146.66,0.5526
LinearModel_Optimizado,1169.60,0.5282,1146.60,0.5526


Se puede ver que no hay una mejora real en las métricas por lo que voy a evaluar otras arquitecturas para mejorar los resultados.

In [28]:
models = {
    'ElasticNet':ElasticNet(),
    'RandomForestRegressor':RandomForestRegressor(),
    'GradientBoostingRegressor':GradientBoostingRegressor()
}

fit_models = {}
for algo, pipeline in models.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

for algo, model in fit_models.items():
    evaluation_score(algo,model)

metrics_models

Métricas del Modelo: *ElasticNet*
ENTRENAMIENTO: RMSE: 1362.23 - R2: 0.36
VALIDACIÓN: RMSE: 1343.04 - R2: 0.3862 

Métricas del Modelo: *RandomForestRegressor*
ENTRENAMIENTO: RMSE: 461.34 - R2: 0.9266
VALIDACIÓN: RMSE: 1158.95 - R2: 0.543 

Métricas del Modelo: *GradientBoostingRegressor*
ENTRENAMIENTO: RMSE: 1085.17 - R2: 0.5939
VALIDACIÓN: RMSE: 1096.04 - R2: 0.5912 



,mse_train,R2_train,mse_val,R2_val
LinearRegression,1169.35,0.5284,1146.66,0.5526
LinearModel_Optimizado,1169.60,0.5282,1146.60,0.5526
ElasticNet,1362.23,0.3600,1343.04,0.3862
RandomForestRegressor,461.34,0.9266,1158.95,0.5430
GradientBoostingRegressor,1085.17,0.5939,1096.04,0.5912


Me quedo con el método 'Gradient Boosting Regressor' ya que tiene las mejores métricas en el set de validación

Ahora con Optuna busco la optimización del nuevo método.

In [11]:
def objective(trial):
    """
    Objective function for optimizing hyperparameters of a Gradient Boosting Regressor using Optuna.
    
    Args:
        trial: A `Trial` object from Optuna that contains the state of the optimization trial.
    
    Returns:
        The mean_squared_error of the validation set using the hyperparameters suggested by Optuna.
    """
    loss = trial.suggest_categorical("loss", ['squared_error', 'absolute_error', 'huber'])
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    criterion = trial.suggest_categorical("criterion", ['friedman_mse', 'squared_error'])
    max_depth = trial.suggest_int('max_depth', 1, 10)
    if loss == 'huber':
        alpha = trial.suggest_uniform('alpha', 0.0, 1.0)
    else:
        alpha = 0.9
    
    
    # Crear el clasificador con los hiperparámetros sugeridos por Optuna
    model = GradientBoostingRegressor(
                                    loss = loss,
                                    learning_rate = learning_rate,
                                    n_estimators = n_estimators,
                                    criterion = criterion,
                                    max_depth = max_depth,
                                    alpha=alpha
                                    )
    
    # Entrenar el clasificador y calcular la precisión en el conjunto de prueba
    model.fit(x_train, y_train)
    pred = model.predict(x_val)
    mse_val = (metrics.mean_squared_error(y_val, pred))**0.5


    return mse_val

# Crear el estudio de Optuna y ejecutar la optimización

sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Imprimir los resultados de la optimización
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

[I 2023-08-04 19:23:48,269] A new study created in memory with name: no-name-6c1435ab-dec5-404d-9c50-5d83c336449c
[I 2023-08-04 19:23:49,423] Trial 0 finished with value: 1138.5756258751358 and parameters: {'loss': 'squared_error', 'learning_rate': 0.024812863342950006, 'n_estimators': 62, 'criterion': 'friedman_mse', 'max_depth': 8}. Best is trial 0 with value: 1138.5756258751358.
[I 2023-08-04 19:23:49,667] Trial 1 finished with value: 1494.6137251375758 and parameters: {'loss': 'absolute_error', 'learning_rate': 0.017434467481818917, 'n_estimators': 30, 'criterion': 'squared_error', 'max_depth': 2}. Best is trial 0 with value: 1138.5756258751358.
[I 2023-08-04 19:23:58,237] Trial 2 finished with value: 1157.9566294915676 and parameters: {'loss': 'huber', 'learning_rate': 0.07472174566050759, 'n_estimators': 165, 'criterion': 'friedman_mse', 'max_depth': 7, 'alpha': 0.8914882120144386}. Best is trial 0 with value: 1138.5756258751358.
[I 2023-08-04 19:23:59,045] Trial 3 finished with 

Best trial: score 1092.4955225928672, params {'loss': 'squared_error', 'learning_rate': 0.0598952381259134, 'n_estimators': 156, 'criterion': 'squared_error', 'max_depth': 2}


In [12]:
optuna.visualization.plot_optimization_history(study)

In [13]:
optuna.visualization.plot_parallel_coordinate(study)

In [14]:
optuna.visualization.plot_param_importances(study)

In [29]:
# Modelo final
GBR_model_opt = GradientBoostingRegressor(**study.best_params)
GBR_model_opt.fit(x_train, y_train)

# Predicción del modelo ajustado para el conjunto de validación
pred = GBR_model_opt.predict(x_val)

# Evaluo el modelo
evaluation_score('Gradient Boosting Regressor_Optimizado',GBR_model_opt)

Métricas del Modelo: *Gradient Boosting Regressor_Optimizado*
ENTRENAMIENTO: RMSE: 1113.64 - R2: 0.5723
VALIDACIÓN: RMSE: 1092.5 - R2: 0.5939 



In [30]:
metrics_models

,mse_train,R2_train,mse_val,R2_val
LinearRegression,1169.35,0.5284,1146.66,0.5526
LinearModel_Optimizado,1169.60,0.5282,1146.60,0.5526
ElasticNet,1362.23,0.3600,1343.04,0.3862
RandomForestRegressor,461.34,0.9266,1158.95,0.5430
GradientBoostingRegressor,1085.17,0.5939,1096.04,0.5912
Gradient Boosting Regressor_Optimizado,1113.64,0.5723,1092.50,0.5939


Comparo las métricas entre el modelo original y el modelo Gradient Boosting Regressor optimizado con Optuna

In [31]:
improve_mse_train = metrics_models['mse_train']['LinearRegression']/metrics_models['mse_train']['Gradient Boosting Regressor_Optimizado']
improve_R2_train = metrics_models['R2_train']['Gradient Boosting Regressor_Optimizado']/metrics_models['R2_train']['LinearRegression']
improve_mse_val = metrics_models['mse_val']['LinearRegression']/metrics_models['mse_val']['Gradient Boosting Regressor_Optimizado']
improve_R2_val = metrics_models['R2_val']['Gradient Boosting Regressor_Optimizado']/metrics_models['R2_val']['LinearRegression']

print(f'La mejora de mse_train fue de: {improve_mse_val-1:.2%}', '\n')
print(f'La mejora de R2_train fue de: {improve_R2_train-1:.2%}', '\n')
print(f'La mejora de mse_val fue de: {improve_mse_val-1:.2%}', '\n')
print(f'La mejora de R2_val fue de: {improve_R2_val-1:.2%}', '\n')

La mejora de mse_train fue de: 4.96% 

La mejora de R2_train fue de: 8.31% 

La mejora de mse_val fue de: 4.96% 

La mejora de R2_val fue de: 7.47% 



Comparo las métricas entre el modelo Gradient Boosting Regressor y el modelo optimizado con Optuna

In [32]:
improve_mse_train = metrics_models['mse_train']['GradientBoostingRegressor']/metrics_models['mse_train']['Gradient Boosting Regressor_Optimizado']
improve_R2_train = metrics_models['R2_train']['Gradient Boosting Regressor_Optimizado']/metrics_models['R2_train']['GradientBoostingRegressor']
improve_mse_val = metrics_models['mse_val']['GradientBoostingRegressor']/metrics_models['mse_val']['Gradient Boosting Regressor_Optimizado']
improve_R2_val = metrics_models['R2_val']['Gradient Boosting Regressor_Optimizado']/metrics_models['R2_val']['GradientBoostingRegressor']

print(f'La mejora de mse_train fue de: {improve_mse_val-1:.2%}', '\n')
print(f'La mejora de R2_train fue de: {improve_R2_train-1:.2%}', '\n')
print(f'La mejora de mse_val fue de: {improve_mse_val-1:.2%}', '\n')
print(f'La mejora de R2_val fue de: {improve_R2_val-1:.2%}', '\n')

La mejora de mse_train fue de: 0.32% 

La mejora de R2_train fue de: -3.64% 

La mejora de mse_val fue de: 0.32% 

La mejora de R2_val fue de: 0.46% 



### Aplico el nuevo modelo en el dataset de test

In [18]:
# Predicción del modelo ajustado
data_test = df_test.copy()
data_test['pred_Sales'] = GBR_model_opt.predict(data_test)
data_test.to_csv('data_test')
data_test.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,pred_Sales
8523,20.750,0.007565,2,21,1,2,0,1,0,0,1766.880186
8524,8.300,0.038428,1,13,0,1,0,1,0,0,1063.145810
8525,14.600,0.099575,4,22,0,0,1,0,0,0,602.832561
8526,7.315,0.015388,3,13,0,1,0,1,0,0,2786.860398
8527,13.600,0.118599,4,35,1,0,0,0,0,1,5616.812635


## Conclusión:

Se puede ver que con el método de 'Gradient Boosting Regressor_Optimizado' tiene una mejora en las métricas de un 5% aproximadamente con respecto al modelo original y la mejora obtenida con Optuna fue del 0.5% aproximadamente.